# Read In Google Sheets Data & Create A PDF


*   This can be done via Google Apps Script, but it can be rated limited (ie too fast and/or script can't run for more than 6 mins
*   Will demonstrate how to read data from a Google Sheets file, produce a single PDF, produce many PDFs + log time, run the process on a set cadence (ie daily



## Install Libraries

In [ ]:
pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.6 MB/s eta 0:00:00


In [ ]:
pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=96c08a13a5d54c20163c032959056e044decbdc18e79b699ac816a27b9f4872d
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


Read In Data From Google Sheets

In [ ]:
#import libraries

import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

from google.colab import drive
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

from fpdf import FPDF
from datetime import date, datetime
import os

#Credtionals
auth.authenticate_user()
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:

# Open worksheets
sh = gc.open_by_url('Google Sheets URL')
worksheet_stock_list = sh.worksheet("Stock List")
worksheet_ohlc = sh.worksheet("OHLC")

# Read data into Pandas DataFrames
data_stock_list = worksheet_stock_list.get_all_values()
df_stock_list = pd.DataFrame(data_stock_list[1:], columns=data_stock_list[0])

data_ohlc = worksheet_ohlc.get_all_values()
df_ohlc = pd.DataFrame(data_ohlc[1:], columns=data_ohlc[0])


In [ ]:
# Display DataFrames I
df_stock_list.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
# Display DataFrames II
df_ohlc.head()

,Symbol,Date,Open,High,Low,Close
0,MMM,11/27/2023,96,97.459999,95.940002,97.07
1,MMM,11/28/2023,96.919998,99.099998,96.540001,98.510002
2,MMM,11/29/2023,99.25,99.449997,98.120003,98.470001
3,MMM,11/30/2023,98.639999,100.080002,98.260002,99.07
4,MMM,12/1/2023,98.839996,100.209999,98.160004,99.849998


Creating The PDF
* Drive must be mounted as part of process

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


Base Example: Single PDF (uncomment all to test/run)

In [ ]:
class PDF(FPDF):
    def __init__(self, symbol, today):
        super().__init__()
        self.symbol = symbol
        self.today = today

    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, f'1 Pager Summary For {self.symbol}', 0, 1, 'C')
        self.cell(0, 10, f'Today\'s Date: {self.today}', 0, 1, 'C')

    def chapter_title(self, title, bold=True, no_border=False):
        self.set_font('Arial', 'B' if bold else '', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(4)

    def chapter_body(self, body, col_widths=None, no_border=False, transpose=False, is_ohlc=False):
        # Transpose the table if specified
        if transpose:
            body = list(map(list, zip(*body)))

        # If column widths are not provided, calculate them
        if col_widths is None:
            col_widths = [max(self.get_string_width(str(col)) for col in row) for row in body]

        row_height = self.font_size
        for row_num, row in enumerate(body):
            for col_num, (col, width) in enumerate(zip(row, col_widths)):
                # Apply bold font to headers of the OHLC table
                font_style = 'B' if is_ohlc and row_num == 0 else ''
                self.set_font('Arial', font_style, 12)

                # Set border style based on the no_border flag
                border_style = '' if no_border else 'LTRB'

                # Format 'Open', 'High', 'Low', 'Close' columns for numeric values
                if is_ohlc and col_num in [1, 2, 3, 4] and row_num > 0:
                    try:
                        col = f"{float(col):,.2f}"
                    except ValueError:
                        pass  # Skip formatting if the value is not numeric

                self.cell(width, row_height, col, border=border_style)

            self.ln(row_height)

        self.ln(10)  # Add a line break after each chapter

def run_process_for_symbol(symbol, df_stock_list, df_ohlc):
    today = date.today().strftime("%Y-%m-%d")

    # Filter data for the specified symbol (Stock List) and transpose it
    # Filter data for the specified symbol (Stock List) and transpose it
    stock_list_data = df_stock_list[df_stock_list['Symbol'] == symbol].values.tolist()


    # Create a PDF document
    pdf = PDF(symbol, today)
    pdf.add_page()

    # Stock List table (transposed) with adjusted column width and no borders
    pdf.chapter_title('Stock List', no_border=True)
    column_widths_stock_list = [60, 90]  # Adjust these widths based on your data
    pdf.chapter_body([df_stock_list.columns.tolist()] + stock_list_data, col_widths=column_widths_stock_list, no_border=True, transpose=True)

    # OHLC table with the correct order of columns, transposed, and the first column dropped
    pdf.chapter_title('OHLC')
    ohlc_data = df_ohlc[df_ohlc['Symbol'] == symbol][['Date', 'Open', 'High', 'Low', 'Close']].values.tolist()
    ohlc_column_widths = [25, 35, 25, 25, 25]  # Adjusted column widths for OHLC
    pdf.chapter_body([['Date', 'Open', 'High', 'Low', 'Close']] + ohlc_data, col_widths=ohlc_column_widths, no_border=False, is_ohlc=True)

    # Save PDF to folder
    output_folder = 'Foler Path'

    # Create the folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    pdf_path = os.path.join(output_folder, f'{symbol}_Summary.pdf')
    pdf.output(pdf_path)

    return pdf_path




# # Example usage
# pdf_path = run_process_for_symbol('MSFT', df_stock_list, df_ohlc)
# print(f"PDF generated and saved at: {pdf_path}")


# Creating Many PDFs


*   Create a PDF for each ticker
*   Log how long process takes



In [ ]:
def run_process_for_all_symbols(df_stock_list, df_ohlc):
    start_time = datetime.now()

    # Get unique symbols from the stock list
    symbols = df_stock_list['Symbol'].unique()

    # Run the process for each symbol
    for symbol in symbols:
        try:
            pdf_path = run_process_for_symbol(symbol, df_stock_list, df_ohlc)
            print(f"PDF generated and saved at: {pdf_path}")
        except UnicodeEncodeError as e:
            print(f"Error generating PDF for symbol {symbol}: {e}")
            continue

    end_time = datetime.now()
    elapsed_time = end_time - start_time
    print(f"Total elapsed time: {elapsed_time}")

# Example usage
run_process_for_all_symbols(df_stock_list, df_ohlc)

PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/MMM_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/AOS_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/ABT_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/ABBV_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/ACN_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/ADM_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/ADBE_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/ADP_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/AES_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/AFL_Summary.pdf
PDF generated and saved at: /content/drive/MyDrive/Stock PDFs/2023-12-06/A_Summary.pdf
PDF generated and sav